In [128]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import requests
import zipfile
import io
import re   
import calendar
import openpyxl

In [129]:
URL = "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela7060.xlsx&terr=N&rank=-&query=t/7060/n1/all/v/all/p/all/c315/7169,7170,7445,7486,7558,7625,7660,7712,7766,7786/d/v63%202,v66%204,v69%202,v2265%202/l/,v%2Bt%2Bp,c315"

r = requests.get(URL)
r = r.content

In [130]:
raw = pd.ExcelFile(r)
raw = pd.read_excel(raw)
raw.head

<bound method NDFrame.head of    Tabela 7060 - IPCA - Variação mensal, acumulada no ano, acumulada em 12 meses e peso mensal, para o índice geral, grupos, subgrupos, itens e subitens de produtos e serviços (a partir de janeiro/2020)  \
0              Geral, grupo, subgrupo, item e subitem                                                                                                                                                        
1                                                 NaN                                                                                                                                                        
2                                                 NaN                                                                                                                                                        
3                                                 NaN                                                                                             

In [131]:
categoria = raw.iloc[4:14, 0]
categoria = categoria.rename()
categoria = categoria.reset_index(drop=True)
categoria_str = []
for i in categoria:
    cat = i.split(".")
    cat_pass = []
    for i in cat:
        if not i.isnumeric():
            cat_pass.append(i)
            
    categoria_str.append(cat_pass)

categoria = [i[0] for i in categoria_str]
categoria = pd.Series(categoria)
categoria

0                 Índice geral
1        Alimentação e bebidas
2                    Habitação
3        Artigos de residência
4                    Vestuário
5                  Transportes
6    Saúde e cuidados pessoais
7            Despesas pessoais
8                     Educação
9                  Comunicação
dtype: object

In [132]:
last_time = raw.iloc[3, 1:]
num_month = int(len(last_time)/4)
last_time = pd.DataFrame(last_time)
last_time.columns = ["mes"]
last_time.reset_index()
last_time = last_time.query("mes == 'janeiro 2020'")
#print(last_time)
last_time = last_time.iloc[1, :]
last_time = last_time.name
last_time = int(last_time.split(" ")[1])
last_time, num_month

(15, 14)

In [133]:
time = raw.iloc[3, 1:last_time]
time = time.reset_index(drop=True)
time

0       janeiro 2020
1     fevereiro 2020
2         março 2020
3         abril 2020
4          maio 2020
5         junho 2020
6         julho 2020
7        agosto 2020
8      setembro 2020
9       outubro 2020
10     novembro 2020
11     dezembro 2020
12      janeiro 2021
13    fevereiro 2021
Name: 3, dtype: object

In [134]:
ipca_mes = raw.iloc[4:(last_time - 1), 1:last_time]
ipca_mes = ipca_mes.reset_index(drop=True)
ipca_mes.columns = time
ipca_mes

3,janeiro 2020,fevereiro 2020,março 2020,abril 2020,maio 2020,junho 2020,julho 2020,agosto 2020,setembro 2020,outubro 2020,novembro 2020,dezembro 2020,janeiro 2021,fevereiro 2021
0,0.21,0.25,0.07,-0.31,-0.38,0.26,0.36,0.24,0.64,0.86,0.89,1.35,0.25,0.86
1,0.39,0.11,1.13,1.79,0.24,0.38,0.01,0.78,2.28,1.93,2.54,1.74,1.02,0.27
2,0.55,-0.39,0.13,-0.1,-0.25,0.04,0.8,0.36,0.37,0.36,0.44,2.88,-1.07,0.4
3,-0.07,-0.08,-1.08,-1.37,0.58,1.3,0.9,0.56,1,1.53,0.86,1.76,0.86,0.66
4,-0.48,-0.73,0.21,0.1,-0.58,-0.46,-0.52,-0.78,0.37,1.11,0.07,0.59,-0.07,0.38
5,0.32,-0.23,-0.9,-2.66,-1.9,0.31,0.78,0.82,0.7,1.19,1.33,1.36,0.41,2.28
6,-0.32,0.73,0.21,-0.22,-0.1,0.35,0.44,0.5,-0.64,0.28,-0.13,0.4,0.32,0.62
7,0.35,0.31,-0.23,-0.14,-0.04,-0.05,-0.11,-0.01,0.09,0.19,0.01,0.65,0.39,0.17
8,0.16,3.7,0.59,0,0.02,0.05,-0.12,-3.47,-0.09,-0.04,-0.02,0.48,0.13,2.48
9,0.12,0.21,0.04,-0.2,0.24,0.75,0.51,0.67,0.15,0.21,0.29,0.39,0.02,-0.13


In [135]:
ipca_ano = raw.iloc[4:(last_time - 1), last_time:(last_time + num_month)]
ipca_ano = ipca_ano.reset_index(drop=True)
ipca_ano.columns = time
ipca_ano

3,janeiro 2020,fevereiro 2020,março 2020,abril 2020,maio 2020,junho 2020,julho 2020,agosto 2020,setembro 2020,outubro 2020,novembro 2020,dezembro 2020,janeiro 2021,fevereiro 2021
0,0.21,0.46,0.53,0.22,-0.16,0.1,0.46,0.7,1.34,2.22,3.13,4.52,0.25,1.11
1,0.39,0.5,1.63,3.45,3.7,4.09,4.1,4.91,7.3,9.37,12.14,14.09,1.02,1.3
2,0.55,0.15,0.28,0.18,-0.07,-0.04,0.76,1.12,1.5,1.86,2.3,5.25,-1.07,-0.67
3,-0.07,-0.15,-1.23,-2.58,-2.02,-0.74,0.16,0.72,1.73,3.28,4.17,6,0.86,1.52
4,-0.48,-1.21,-1,-0.9,-1.48,-1.94,-2.44,-3.21,-2.85,-1.77,-1.7,-1.13,-0.07,0.31
5,0.32,0.1,-0.8,-3.44,-5.28,-4.99,-4.25,-3.46,-2.79,-1.63,-0.32,1.03,0.41,2.7
6,-0.32,0.41,0.62,0.39,0.3,0.65,1.09,1.6,0.95,1.23,1.1,1.5,0.32,0.95
7,0.35,0.66,0.43,0.29,0.25,0.2,0.1,0.08,0.17,0.36,0.38,1.03,0.39,0.56
8,0.16,3.86,4.47,4.47,4.49,4.54,4.42,0.79,0.7,0.66,0.64,1.13,0.13,2.61
9,0.12,0.33,0.36,0.16,0.4,1.15,1.67,2.35,2.5,2.72,3.02,3.42,0.02,-0.11


In [136]:

ipca_12m = raw.iloc[4:(last_time - 1), (last_time + num_month):(last_time + 2 * num_month)]
ipca_12m = ipca_12m.reset_index(drop=True)
ipca_12m.columns = time
ipca_12m

3,janeiro 2020,fevereiro 2020,março 2020,abril 2020,maio 2020,junho 2020,julho 2020,agosto 2020,setembro 2020,outubro 2020,novembro 2020,dezembro 2020,janeiro 2021,fevereiro 2021
0,...,...,...,...,...,...,...,...,...,...,...,4.52,4.56,5.2
1,...,...,...,...,...,...,...,...,...,...,...,14.09,14.81,15
2,...,...,...,...,...,...,...,...,...,...,...,5.25,3.56,4.39
3,...,...,...,...,...,...,...,...,...,...,...,6,6.98,7.78
4,...,...,...,...,...,...,...,...,...,...,...,-1.13,-0.72,0.39
5,...,...,...,...,...,...,...,...,...,...,...,1.03,1.12,3.66
6,...,...,...,...,...,...,...,...,...,...,...,1.5,2.15,2.05
7,...,...,...,...,...,...,...,...,...,...,...,1.03,1.07,0.93
8,...,...,...,...,...,...,...,...,...,...,...,1.13,1.1,-0.09
9,...,...,...,...,...,...,...,...,...,...,...,3.42,3.32,2.98


In [137]:
peso = raw.iloc[4:(last_time - 1), (last_time + 2 * num_month):(last_time + 3 * num_month)]
peso = peso.reset_index(drop=True)
peso.columns = time

table_peso = pd.concat([categoria, peso], axis=1)
table_peso = table_peso.iloc[1:,:]
table_peso = table_peso.rename(columns={0: "Categoria"})

table_peso

,Categoria,janeiro 2020,fevereiro 2020,março 2020,abril 2020,maio 2020,junho 2020,julho 2020,agosto 2020,setembro 2020,outubro 2020,novembro 2020,dezembro 2020,janeiro 2021,fevereiro 2021
1,Alimentação e bebidas,19.3483,19.3816,19.3545,19.5588,19.9696,20.0937,20.1194,20.0474,20.1553,20.4823,20.6985,21.037,21.1159,21.2779
2,Habitação,15.5945,15.6461,15.547,15.5544,15.5855,15.6048,15.5709,15.6402,15.6597,15.6198,15.5426,15.4721,15.7065,15.4995
3,Artigos de residência,3.7529,3.7423,3.7298,3.6874,3.6487,3.6841,3.7222,3.742,3.7537,3.7668,3.7917,3.7903,3.8053,3.8281
4,Vestuário,4.577,4.5455,4.5008,4.5075,4.5263,4.5173,4.4849,4.4449,4.3994,4.3868,4.3974,4.3621,4.3285,4.3145
5,Transportes,20.5979,20.6217,20.5255,20.3299,19.8534,19.5495,19.5577,19.6398,19.7519,19.7627,19.8271,19.9125,19.9153,19.9472
6,Saúde e cuidados pessoais,13.5334,13.4612,13.5247,13.5438,13.5533,13.5915,13.6048,13.6152,13.6502,13.478,13.4007,13.2657,13.1419,13.1514
7,Despesas pessoais,10.733,10.7474,10.7541,10.7233,10.7427,10.7785,10.7453,10.6963,10.6677,10.6119,10.5415,10.4488,10.3787,10.3935
8,Educação,6.1485,6.1448,6.356,6.3893,6.4098,6.4346,6.4198,6.3903,6.1531,6.1103,6.0558,6.0007,5.9514,5.9442
9,Comunicação,5.7146,5.7096,5.7077,5.7055,5.7106,5.7461,5.7748,5.7838,5.8089,5.7815,5.7448,5.7108,5.6564,5.6437


In [138]:
ipca_mes_last = ipca_mes.iloc[:, -1]
ipca_ano_last = ipca_ano.iloc[:, -1]
ipca_12m_last = ipca_12m.iloc[:, -1]
table_main = pd.concat([categoria, ipca_mes_last, ipca_ano_last, ipca_12m_last], axis=1)
table_main.columns = ["", ipca_mes_last.name, "Acumulado no ano", "Acumulado 12 meses"]
table_main

,,fevereiro 2021,Acumulado no ano,Acumulado 12 meses
0,Índice geral,0.86,1.11,5.2
1,Alimentação e bebidas,0.27,1.3,15
2,Habitação,0.4,-0.67,4.39
3,Artigos de residência,0.66,1.52,7.78
4,Vestuário,0.38,0.31,0.39
5,Transportes,2.28,2.7,3.66
6,Saúde e cuidados pessoais,0.62,0.95,2.05
7,Despesas pessoais,0.17,0.56,0.93
8,Educação,2.48,2.61,-0.09
9,Comunicação,-0.13,-0.11,2.98


In [139]:
URL_HIST = "https://ftp.ibge.gov.br/Precos_Indices_de_Precos_ao_Consumidor/IPCA/Serie_Historica/ipca_SerieHist.zip"

hist_raw = requests.get(URL_HIST, stream=True)
hist_raw = zipfile.ZipFile(io.BytesIO(hist_raw.content))

hist_raw_filename = str(hist_raw.filelist[0])
hist_raw_filename = hist_raw_filename.split("'")[1]

hist_raw = hist_raw.read(hist_raw_filename)
hist_raw = pd.ExcelFile(hist_raw)
hist_raw = pd.read_excel(hist_raw)
hist_raw


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,SÉRIE HISTÓRICA DO IPCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(continua)
2,NaN,NaN,NaN,NaN,NaN,VARIAÇÃO,NaN,NaN
3,ANO,MÊS,NÚMERO ÍNDICE,(%),NaN,NaN,NaN,NaN
4,NaN,NaN,(DEZ 93 = 100),NO,3,6,NO,12
...,...,...,...,...,...,...,...,...
413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
hist = hist_raw.iloc[7:len(hist_raw), [0,1,3]]
hist = hist.dropna(how="all")

#hist[0] = hist[0].fillna(method="ffill")
hist.iloc[:,0] = hist.iloc[:,0].fillna(method="ffill")
hist = hist.dropna()
hist.columns = ["ano", "mes", "valor"]
hist = hist[hist.valor != "(%)"]
hist = hist.reset_index(drop=True)
hist

,ano,mes,valor
0,1994,JAN,41.31
1,1994,FEV,40.27
2,1994,MAR,42.75
3,1994,ABR,42.68
4,1994,MAI,44.03
...,...,...,...
321,2020,OUT,0.86
322,2020,NOV,0.89
323,2020,DEZ,1.35
324,2021,JAN,0.25


In [141]:
for i in range(len(hist)):
    print(list(hist.iloc[i,:]))

[1994, 'JAN', 41.31]
[1994, 'FEV', 40.27]
[1994, 'MAR', 42.75]
[1994, 'ABR', 42.68]
[1994, 'MAI', 44.03]
[1994, 'JUN', 47.43]
[1994, 'JUL', 6.84]
[1994, 'AGO', 1.86]
[1994, 'SET', 1.53]
[1994, 'OUT', 2.62]
[1994, 'NOV', 2.81]
[1994, 'DEZ', 1.71]
[1995, 'JAN', 1.7]
[1995, 'FEV', 1.02]
[1995, 'MAR', 1.55]
[1995, 'ABR', 2.43]
[1995, 'MAI', 2.67]
[1995, 'JUN', 2.26]
[1995, 'JUL', 2.36]
[1995, 'AGO', 0.99]
[1995, 'SET', 0.99]
[1995, 'OUT', 1.41]
[1995, 'NOV', 1.47]
[1995, 'DEZ', 1.56]
[1996, 'JAN', 1.34]
[1996, 'FEV', 1.03]
[1996, 'MAR', 0.35]
[1996, 'ABR', 1.26]
[1996, 'MAI', 1.22]
[1996, 'JUN', 1.19]
[1996, 'JUL', 1.11]
[1996, 'AGO', 0.44]
[1996, 'SET', 0.15]
[1996, 'OUT', 0.3]
[1996, 'NOV', 0.32]
[1996, 'DEZ', 0.47]
[1997, 'JAN', 1.18]
[1997, 'FEV', 0.5]
[1997, 'MAR', 0.51]
[1997, 'ABR', 0.88]
[1997, 'MAI', 0.41]
[1997, 'JUN', 0.54]
[1997, 'JUL', 0.22]
[1997, 'AGO', -0.02]
[1997, 'SET', 0.06]
[1997, 'OUT', 0.23]
[1997, 'NOV', 0.17]
[1997, 'DEZ', 0.43]
[1998, 'JAN', 0.71]
[1998, 'FEV', 0.

In [142]:
data_str = [str(hist.iloc[i,1]) + "-" + str(hist.iloc[i, 0]) for i in range(len(hist.mes))]
data_str = pd.Series(data_str)
data_str

0      JAN-1994
1      FEV-1994
2      MAR-1994
3      ABR-1994
4      MAI-1994
         ...   
321    OUT-2020
322    NOV-2020
323    DEZ-2020
324    JAN-2021
325    FEV-2021
Length: 326, dtype: object

In [143]:
table_hist = pd.concat([data_str, hist.iloc[:,2]], axis=1)
table_hist.columns = ["Data", "IPCA no Mês"]
table_hist
#for i in range(len(table_hist)):
#    print(list(table_hist.iloc[i,:]))

,Data,IPCA no Mês
0,JAN-1994,41.31
1,FEV-1994,40.27
2,MAR-1994,42.75
3,ABR-1994,42.68
4,MAI-1994,44.03
...,...,...
321,OUT-2020,0.86
322,NOV-2020,0.89
323,DEZ-2020,1.35
324,JAN-2021,0.25


In [144]:

# In[34]:


with pd.ExcelWriter("RENAN STELLET IPCA.xlsx") as writer:
    table_main.to_excel(writer, index=False, sheet_name="Tabela")
    table_hist.to_excel(writer, index=False, sheet_name="IPCA mensal histórico")
    table_peso.to_excel(writer, index=False, sheet_name="Pesos")

workbook = openpyxl.load_workbook("RENAN STELLET IPCA.xlsx")

xcl_main = workbook["Tabela"]
xcl_main.column_dimensions["A"].width = 24
xcl_main.column_dimensions["B"].width = 10
xcl_main.column_dimensions["C"].width = 15
xcl_main.column_dimensions["D"].width = 15
xcl_main.row_dimensions[1].height = 30

xcl_main['B1'].alignment = openpyxl.styles.Alignment(horizontal="center", vertical="center", wrapText=True)
xcl_main['C1'].alignment = openpyxl.styles.Alignment(horizontal="center", vertical="center", wrapText=True)
xcl_main['D1'].alignment = openpyxl.styles.Alignment(horizontal="center", vertical="center", wrapText=True)

xcl_peso = workbook["Pesos"]
xcl_peso.column_dimensions["A"].width = 25
xcl_peso_columntofit = ["B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O"]
for i in xcl_peso_columntofit:
    xcl_peso.column_dimensions[i].width = 15

xcl_hist = workbook["IPCA mensal histórico"]
xcl_hist.column_dimensions["A"].width = 12
xcl_hist.column_dimensions["B"].width = 12

workbook.save("RENAN STELLET IPCA.xlsx")
